In [1]:
# Core
import numpy as np
import pandas as pd

# MSE
import kepler_sieve
from asteroid_dataframe import load_ast_data, calc_ast_data, spline_ast_vec_df, calc_ast_dir
from candidate_element import asteroid_elts
from ztf_data import load_ztf_det_all, ztf_reindex
from ztf_ast import load_ztf_nearest_ast, calc_hit_freq
from ztf_element import make_ztf_near_elt
from astro_utils import deg2dist

# Typing
from typing import Dict

## Load Data for First 64 Asteroids

In [2]:
# Asteroids to load
n0 = 1
n1 = n0+64
# Manually specify date range
mjd0 = 58270.0
mjd1 = 58906.0
progbar=True

In [3]:
# Load data in this date range
df_ast_load, df_earth_load, df_sun_load = load_ast_data(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, progbar=progbar)

Loading asteroid data from n0=1 to n1=65 in 1 blocks...


In [4]:
# df_ast_load

In [5]:
# Orbital elements for first 64 asteroids (dict of numpy arrays)
asteroid_nums = np.arange(n0, n1, dtype=np.int32)
elts = asteroid_elts(asteroid_nums)

In [6]:
# Calculate vectors from these elements on the fly with calc_ast_data
df_ast_calc, df_earth_calc, df_sun_calc = calc_ast_data(elts=elts, mjd0=mjd0, mjd1=mjd1, element_id=asteroid_nums)

In [7]:
# df_ast_calc

In [8]:
# Check agreement
cols_q = ['qx', 'qy', 'qz']
cols_v = ['vx', 'vy', 'vz']

dq_ast = df_ast_calc[cols_q] - df_ast_load[cols_q]
dv_ast = df_ast_calc[cols_v] - df_ast_load[cols_v]

dq_earth = df_earth_calc[cols_q] - df_earth_load[cols_q]
dv_earth = df_earth_calc[cols_v] - df_earth_load[cols_v]

dq_sun = df_sun_calc[cols_q] - df_sun_load[cols_q]
dv_sun = df_sun_calc[cols_v] - df_sun_load[cols_v]

# Report
# print(f'mean position difference = {np.mean(np.linalg.norm(dq, axis=1)):6.2e}')
print('Mean position difference in AU:')
print(f'asteroids: {np.mean(np.linalg.norm(dq_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dq_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dq_sun, axis=1)):6.2e}')

print('\nMean velocity difference in AU/day:')
print(f'asteroids: {np.mean(np.linalg.norm(dv_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dv_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dv_sun, axis=1)):6.2e}')

Mean position difference in AU:
asteroids: 1.87e-12
earth    : 3.07e-12
sun      : 1.52e-15

Mean velocity difference in AU/day:
asteroids: 7.79e-15
earth    : 5.31e-14
sun      : 1.92e-18


## Splined Direction at ZTF Dates

In [9]:
# All the ZTF detections
ztf, mjd_unq = load_ztf_det_all()

In [10]:
# ztf

In [11]:
# mjd for the spline output
mjd = mjd_unq

In [12]:
# Spline the first 64 asteroids at the unique ZTF dates using the loaded asteroid data
df_ast_load_out, df_earth_load_out, df_sun_load_out = spline_ast_vec_df(df_ast=df_ast_load, df_earth=df_earth_load, df_sun=df_sun_load, mjd=mjd, include_elts=False)

In [13]:
# df_ast_load_out

In [14]:
# Spline the first 64 asteroids at the unique ZTF dates using the calculated asteroid data
df_ast_calc_out, df_earth_calc_out, df_sun_calc_out = spline_ast_vec_df(df_ast=df_ast_calc, df_earth=df_earth_calc, df_sun=df_sun_calc, mjd=mjd, include_elts=False)

In [15]:
# df_ast_calc_out

In [16]:
# Check agreement
cols_q = ['qx', 'qy', 'qz']
cols_v = ['vx', 'vy', 'vz']

dq_ast = df_ast_calc_out[cols_q] - df_ast_load_out[cols_q]
dv_ast = df_ast_calc_out[cols_v] - df_ast_load_out[cols_v]

dq_earth = df_earth_calc_out[cols_q] - df_earth_load_out[cols_q]
dv_earth = df_earth_calc_out[cols_v] - df_earth_load_out[cols_v]

dq_sun = df_sun_calc_out[cols_q] - df_sun_load_out[cols_q]
dv_sun = df_sun_calc_out[cols_v] - df_sun_load_out[cols_v]

# Report
# print(f'mean position difference = {np.mean(np.linalg.norm(dq, axis=1)):6.2e}')
print('Mean position difference in AU:')
print(f'asteroids: {np.mean(np.linalg.norm(dq_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dq_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dq_sun, axis=1)):6.2e}')

print('\nMean velocity difference in AU/day:')
print(f'asteroids: {np.mean(np.linalg.norm(dv_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dv_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dv_sun, axis=1)):6.2e}')

Mean position difference in AU:
asteroids: 3.05e-12
earth    : 5.02e-12
sun      : 2.48e-15

Mean velocity difference in AU/day:
asteroids: 1.27e-14
earth    : 8.67e-14
sun      : 3.13e-18


## Assemble Batch of Asteroids Near Orbital Elements

In [17]:
# Load nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [18]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top 64
idx = np.argsort(hit_count)[::-1][0:64]

# Extract the asteroid number and hit count for this batch
ast_nums_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [19]:
# Orbital elements for best asteroids (DataFrame)
element_id_best = np.sort(ast_nums_best)
elts_best = asteroid_elts(element_id_best)

In [20]:
# Compute mjd0 and mjd1 from mjd_unq
mjd0 = np.floor(np.min(mjd_unq))
mjd1 = np.ceil(np.max(mjd_unq))

# Calculate positions in this date range, sampled daily
df_ast_daily, df_earth_daily, df_sun_daily = calc_ast_data(elts=elts_best, mjd0=mjd0, mjd1=mjd1, element_id=element_id_best)

In [21]:
# Spline positions at ztf times
mjd = mjd_unq
df_ast, df_earth, df_sun = spline_ast_vec_df(df_ast=df_ast_daily, df_earth=df_earth_daily, df_sun=df_sun_daily, mjd=mjd, include_elts=False)

In [22]:
# Direction from palomar
df_dir = calc_ast_dir(df_ast=df_ast, df_earth=df_earth, site_name='palomar')

In [23]:
# df_dir

In [24]:
# Calculate subset of ZTF data within threshold of this batch
thresh_deg = 1.0
probar = True
ztf_elt = make_ztf_near_elt(ztf=ztf, df_dir=df_dir, df_ast=df_ast, thresh_deg=thresh_deg, progbar=progbar)

In [25]:
# Choose one of the ZTF data sets
element_test = ast_nums_best[0]
mask_test = (ztf_elt.element_id == element_test)
ztf_i = ztf_elt[mask_test]

In [26]:
ztf_elt

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
0,37606,733,b'ZTF18abtdsvv',621336711115010010,9577,58375.336713,350.405941,0.879484,0.985897,-0.146790,...,0.002999,0.983072,-0.161029,0.087400,2.595640,0.016129,3326.818565,0.999870,8.539940e-01,False
1,37607,733,b'ZTF18abtdsvv',621309991115010019,9572,58375.310000,350.405990,0.879483,0.985897,-0.146789,...,0.002999,0.983090,-0.160937,0.087369,2.595627,0.016032,3306.899244,0.999871,8.437983e-01,False
2,37610,733,b'ZTF18abtdsvv',614358404915010002,8240,58368.358403,350.405972,0.879528,0.985897,-0.146789,...,0.003009,0.987293,-0.137772,0.079197,2.599708,0.009200,1897.647155,0.999958,2.778650e-01,False
3,37612,733,b'ZTF18abtdsvv',615324371115015021,8425,58369.324375,350.405987,0.879540,0.985897,-0.146789,...,0.003008,0.986748,-0.140962,0.080359,2.598268,0.005888,1214.529070,0.999983,1.138205e-01,False
4,141833,733,b'ZTF18abwpfho',623339405515010000,9980,58377.339410,348.764546,0.189321,0.980829,-0.177448,...,0.002996,0.981751,-0.167709,0.089660,2.597194,0.013378,2759.455450,0.999911,5.875520e-01,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90206,5584064,324582,b'ZTF18aaaknvz',1149162620015010022,96198,58903.162627,67.404504,45.304078,0.270241,0.878532,...,-0.002806,0.271240,0.872415,0.406597,1.545974,0.014135,2915.643014,0.999900,6.559451e-01,False
90207,5584155,324582,b'ZTF20aaqjjhn',1149162620115015001,96198,58903.162627,66.164947,45.635778,0.282557,0.871180,...,-0.002806,0.271240,0.872415,0.406597,1.545974,0.012471,2572.259178,0.999922,5.105401e-01,False
90208,5584262,324582,b'ZTF20aaqjltu',1149162621915015008,96198,58903.162627,67.072721,46.398450,0.268658,0.870797,...,-0.002806,0.271240,0.872415,0.406597,1.545974,0.005984,1234.250587,0.999982,1.175470e-01,False
90209,5584879,324582,b'ZTF20aaqjltx',1149162621915015001,96198,58903.162627,66.993445,46.563928,0.268718,0.869478,...,-0.002806,0.271240,0.872415,0.406597,1.545974,0.008788,1812.649841,0.999961,2.535310e-01,False


In [27]:
ztf_i

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
60421,53851,51921,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.010624,2191.408734,0.999944,0.370552,False
60422,73604,51921,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.016809,3467.151428,0.999859,0.927559,False
60423,82343,51921,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,...,0.003825,0.000919,-0.977996,0.208622,2.703478,0.005450,1124.103915,0.999985,0.097503,False
60424,257221,51921,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,...,-0.001953,0.639004,0.610779,0.467571,2.175851,0.008713,1797.091521,0.999962,0.249197,False
60425,327000,51921,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,...,-0.002129,0.606278,0.637608,0.475272,2.114866,0.007949,1639.539679,0.999968,0.207419,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61840,5585430,51921,b'ZTF20aaqjlxm',1149165065915015030,96202,58903.165069,58.775292,21.231723,0.483209,0.875373,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.003645,751.740548,0.999993,0.043606,False
61841,5585539,51921,b'ZTF20aaqjkpu',1149165064115015000,96202,58903.165069,57.817755,20.738491,0.498104,0.867059,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.014227,2934.453801,0.999899,0.664436,False
61842,5585636,51921,b'ZTF20aaqjlxl',1149165065915015014,96202,58903.165069,58.639323,21.532043,0.484105,0.874763,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.007480,1542.796441,0.999972,0.183663,False
61843,5585648,51921,b'ZTF20aaqjkqe',1149165064115015002,96202,58903.165069,58.007009,20.618973,0.495877,0.868362,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.012832,2646.735147,0.999918,0.540532,False


In [28]:
# Build version of ztf_tbl that includes nearest asteroid information
ztf_elt_near = make_ztf_near_elt(ztf=ztf_ast, df_dir=df_dir, df_ast=df_ast, thresh_deg=thresh_deg, progbar=progbar)

/home/michael/IACS/kepler-sieve/src/ztf_element.py:171: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ztf_i.is_match = (ztf_i.nearest_ast_num == ztf_i.element_id)


In [29]:
mask_test_near = (ztf_elt_near.element_id == element_test)
ztf_near_i = ztf_elt_near[mask_test_near]
ztf_near_i

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
60421,53851,51921,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.010624,2191.408734,0.999944,0.370552,False
60422,73604,51921,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.016809,3467.151428,0.999859,0.927559,False
60423,82343,51921,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,...,0.003825,0.000919,-0.977996,0.208622,2.703478,0.005450,1124.103915,0.999985,0.097503,False
60424,257221,51921,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,...,-0.001953,0.639004,0.610779,0.467571,2.175851,0.008713,1797.091521,0.999962,0.249197,False
60425,327000,51921,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,...,-0.002129,0.606278,0.637608,0.475272,2.114866,0.007949,1639.539679,0.999968,0.207419,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61840,5585430,51921,b'ZTF20aaqjlxm',1149165065915015030,96202,58903.165069,58.775292,21.231723,0.483209,0.875373,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.003645,751.740548,0.999993,0.043606,False
61841,5585539,51921,b'ZTF20aaqjkpu',1149165064115015000,96202,58903.165069,57.817755,20.738491,0.498104,0.867059,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.014227,2934.453801,0.999899,0.664436,False
61842,5585636,51921,b'ZTF20aaqjlxl',1149165065915015014,96202,58903.165069,58.639323,21.532043,0.484105,0.874763,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.007480,1542.796441,0.999972,0.183663,False
61843,5585648,51921,b'ZTF20aaqjkqe',1149165064115015002,96202,58903.165069,58.007009,20.618973,0.495877,0.868362,...,-0.006031,0.486101,0.873796,0.013632,2.000131,0.012832,2646.735147,0.999918,0.540532,False


## Test that Near Batch is a Superset of Observations with Matching Nearest Asteroid Number

In [30]:
# Assemble observations nearest to selected asteroid that are also within the threshold
thresh_dist = deg2dist(thresh_deg)
mask_test = (ztf_ast.nearest_ast_num == element_test) & (ztf_ast.nearest_ast_dist < thresh_dist)

# The test subset
ztf_test = ztf_ast[mask_test]

In [31]:
ztf_test

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob,nearest_ast_num,nearest_ast_dist,ast_ra,ast_dec,ast_ux,ast_uy,ast_uz
257221,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,0.470042,19.289200,0.810351,51921,0.008713,30.365848,42.217117,0.639004,0.610779,0.467571
327000,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,0.481893,17.725201,0.991371,51921,0.007949,33.147321,43.604733,0.606278,0.637608,0.475272
336873,b'ZTF19abizrbv',937428210615015010,40798,58691.428218,32.768773,44.192434,0.602900,0.633333,0.485181,19.924500,0.983333,51921,0.011358,33.130141,43.596052,0.606484,0.637450,0.475221
340306,b'ZTF19abiyxfi',937402265115010001,40777,58691.402269,33.223154,43.100462,0.610808,0.638835,0.467765,20.192900,0.841761,51921,0.008648,33.118298,43.590070,0.606626,0.637341,0.475186
341182,b'ZTF19abizrbu',937428210615015008,40798,58691.428218,33.030390,44.317294,0.599847,0.635715,0.485850,19.837000,0.985742,51921,0.012650,33.130141,43.596052,0.606484,0.637450,0.475221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5379997,b'ZTF20aaogcyq',1142172456115015014,94578,58896.172454,55.968493,21.904633,0.519245,0.853848,0.036427,17.839100,1.000000,51921,0.000007,55.968085,21.904751,0.519250,0.853845,0.036431
5450439,b'ZTF20aapdkvt',1145124016315015006,95210,58899.124016,57.062507,21.552337,0.505708,0.862296,0.026539,17.545500,1.000000,51921,0.000007,57.062108,21.552442,0.505713,0.862293,0.026542
5569973,b'ZTF20aaqifhg',1149110835915015022,96107,58903.110833,58.576908,21.111013,0.486363,0.873648,0.013797,18.046301,1.000000,51921,0.000007,58.576520,21.111107,0.486368,0.873645,0.013800
5576802,b'ZTF20aaqiwtw',1149142105915015021,96161,58903.142107,58.588750,21.107654,0.486209,0.873735,0.013700,18.070601,1.000000,51921,0.000007,58.588361,21.107762,0.486214,0.873732,0.013703


In [32]:
# These indices on the test subset should be contained in the subset of ztf close to the elements, ztf_i
index_hit = ztf_test.index

In [33]:
index_hit

Int64Index([ 257221,  327000,  336873,  340306,  341182,  341737,  345725,
             346366,  346522,  347644,
            ...
            5163761, 5182803, 5243662, 5374010, 5379996, 5379997, 5450439,
            5569973, 5576802, 5584540],
           dtype='int64', length=288)